In [1]:
import pandas as pd
import requests
import urllib.request as req
import json
from bs4 import BeautifulSoup
import requests.packages.urllib3
import json
import re

requests.packages.urllib3.disable_warnings() #使用 requests 時發生 InsecureRequestWarning: Unverified HTTPS request is being made

# **爬文**

In [2]:
def contenter(url):
    request = req.Request(url, headers = {
    "cookie": "over18=1",    
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"
    })
    with req.urlopen(request) as response:
        data = response.read().decode("utf-8")
    soup = BeautifulSoup(data, "html.parser")

    main_container = soup.find(id = 'main-container')
    all_text = main_container.text
    pre_texts = all_text.split('※ 發信站')[:-1]
    # 將前面的所有內容合併成一個
    one_text = "--".join(pre_texts)
    one_text
    #url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-zA-Z][0-9a-zA-Z]))+')
    #new_string = url_pattern.sub(r'', new_string)
    #new_string
    # 以\n切割，第一行標題不要
    texts = one_text.split("\n")[1:]
    li_text = []
    for i in texts:
        if ('※' not in i) & (len(i) != 0):
            li_text.append(i)
    content = " ".join(li_text)
    url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-zA-Z][0-9a-zA-Z]))+')
    content = url_pattern.sub(r'', content)
    return content

In [70]:
import time
title = []
date = []
content = []
for page in range(501, 543): #543
    url = 'https://www.ptt.cc/bbs/Gossiping/search?page=' + str(page) +'&q=%E9%99%B3%E6%99%82%E4%B8%AD' 
    request = req.Request(url, headers = {
        "cookie": "over18=1",    
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"
    })
    with req.urlopen(request) as response:
        data = response.read().decode("utf-8")
    
    soup = BeautifulSoup(data, "html.parser")
    titles = soup.find_all('div', class_= 'r-ent')
    dt = soup.find_all('div', class_ = 'date')
    t_d = dict(zip(titles, dt))


    try:
        for t, d in t_d.items():
            if t.a != None:
                title.append(t.a.string)
                date.append(d.string)
                url = "https://www.ptt.cc"+t.a.get("href")
                cont = contenter(url)
                if len(cont) != 0:
                    content.append(cont)
                else:
                    content.append(None)
            else:
                pass
    except:
        print('第{}頁'.format(page))
    time.sleep(5)
    

dic = {
    '標題': title,
    '發布日期': date,
    '內文': content
}


ptt_test_5 = pd.DataFrame(dic)

In [92]:
df = pd.read_csv('C:/Users/CHARLIE/Desktop/文字探勘/期末/陳時中.csv', index_col = False)
df

,Column1,標題,發布日期,內文
0,0,[新聞]又一綠側翼遭起底!10月裝大學生同框陳時中,2022/12/3,作者leocean9816 (阿光)看板Gossiping標題[新聞]又一綠側翼遭起底!10...
1,1,[新聞] 發文感謝支持 陳時中：面對未來嚴峻挑戰,2022/12/1,作者x5723 (自然)看板Gossiping標題[新聞] 發文感謝支持 陳時中：面對未來嚴...
2,2,[新聞] 「陳時中選輸都我害的？」周玉蔻質問民,2022/12/1,作者george303 (ASICS)看板Gossiping標題[新聞] 「陳時中選輸都我害...
3,3,Re: [新聞] 三立開票誤植屏東縣長票數、疑灌水陳時中,2022/12/1,作者YU0121 (愛玉冰™)看板Gossiping標題Re: [新聞] 三立開票誤植屏東縣...
4,4,[新聞] 三立開票誤植屏東縣長票數、疑灌水陳時中,2022/12/1,作者jis0077 (一個人的輕旅行)看板Gossiping標題[新聞] 三立開票誤植屏東縣...
...,...,...,...,...
10830,10830,[問卦] 現在該聽柚子醫生的還是陳時中的,2021/6/12,"[""作者owlman ( ′▽`)-o█ █o-(￣▽￣)看板Gossiping標題[問卦]..."
10831,10831,[政治] 陳時中質疑電話疫調有問題！韓市府：別縣,2021/4/21,['作者sweat992001 (小樽)看板Gossiping標題[政治] 陳時中質疑電話疫...
10832,10832,[政治] 陳時中喊話譚德塞：與其罵台灣 不如向台,2021/4/9,['作者a870209tw ()看板Gossiping標題[政治] 陳時中喊話譚德塞：與其罵...
10833,10833,[政治] 時力民調：提換掉陳時中後 民眾黨表現摔,2021/3/29,['作者indium111 (#ttyhg)看板Gossiping標題[政治] 時力民調：提...


# **整理文章**

In [35]:
ptt_test.drop_duplicates(subset='發布日期', keep='first', inplace=True)
dt = dict(zip(ptt_test['發布日期'], ptt_test['內文']))
review = []
for k, v in dt.items():
    clean_content = v.split(k)[1]
    review.append(clean_content)

In [58]:
ptt_test['內文(clean)'] = review
ptt_test

,標題,發布日期,內文,內文(clean)
0,[新聞] 小女孩姊姊「想領陳時中的市長獎」 陳時,Fri Nov 25 21:07:30 2022,作者zakijudelo (Zack)看板Gossiping標題[新聞] 小女孩姊姊「想領陳...,備註請放最後面 違者新聞文章刪除 1.媒體來源: 自由時報 2.記者署名: 何玉華 3.完...
1,[新聞] 風雨無阻！支持者湧入陳時中選前之夜 騎,Fri Nov 25 19:45:15 2022,作者wppw (123)看板Gossiping標題[新聞] 風雨無阻！支持者湧入陳時中選前之...,備註請放最後面 違者新聞文章刪除 1.媒體來源: 風雨無阻！支持者湧入陳時中選前之夜 騎樓...
2,Re: [新聞] 陳時中再強調高端疫苗的必要性：國家難道,Fri Nov 25 19:40:44 2022,作者ladioshuang (達哥)看板Gossiping標題Re: [新聞] 陳時中再強調...,支持國產疫苗不表示要支持沒用的垃圾 蔡EE一直以來都慣用情緒勒索 將垃圾跟自己綁在一起 跟...
3,Re: [新聞] 陳時中再強調高端疫苗的必要性：國家難道,Fri Nov 25 18:14:52 2022,作者nasa709 (17)看板Gossiping標題Re: [新聞] 陳時中再強調高端疫苗...,其實陳時中目前能做的就選舉來說就是堅持他既往是對的， 把基本盤穩住在穩住不能在流失了， 接...
4,[新聞] 陳時中批柯施政最後一名應換人做做看,Fri Nov 25 17:53:33 2022,作者asdf1256 (guest)看板Gossiping標題[新聞] 陳時中批柯施政最後一...,1.媒體來源: 2.記者署名: 3.完整新聞標題: 4.完整新聞內文: 台北市長柯文哲於選...
...,...,...,...,...
75,[新聞] 陳時中松山信義車隊掃街 沿途遇民眾比倒,Wed Nov 23 16:24:47 2022,作者pkmu8426 (巴426)看板Gossiping標題[新聞] 陳時中松山信義車隊掃街...,UDN 林俊良 陳時中松山信義車隊掃街 沿途遇民眾比倒讚 民進黨台北市長候選人陳時中上午...
76,Re: [新聞] 嘆陳時中被抹黑「惡魔」與「色魔」 他曝,Wed Nov 23 14:15:02 2022,作者a111111 ( )看板Gossiping標題Re: [新聞] 嘆陳時中被抹黑「惡魔」...,陳時中囤房19筆 用剝削百姓的居住權 來發財的無良囤房蟲 這不是惡魔是什麼？ 台灣空房率...
77,[新聞] 陳時中「恐懼年代」故事曝光 曾觸診千名,Wed Nov 23 14:10:16 2022,作者EstelleRinz (小艾)看板Gossiping標題[新聞] 陳時中「恐懼年代」故...,1.媒體來源: ETtoday 2.記者署名: 記者呂晏慈／台北報導 3.完整新聞標題: ...
78,[新聞] 嘆陳時中被抹黑「惡魔」與「色魔」 他曝,Wed Nov 23 14:07:35 2022,作者JAL96 (JAL96)看板Gossiping標題[新聞] 嘆陳時中被抹黑「惡魔」與「...,1.媒體來源:ETtoday 2.記者署名:呂晏慈 3.完整新聞標題:嘆陳時中被抹黑「惡魔...


# **分成新聞內文和鄉民評論**

In [61]:
df_news = ptt_test[~ptt_test['標題'].str.contains('Re:')]
df_review = ptt_test[ptt_test['標題'].str.contains('Re:')]